This notebooks uses the dataset from 'Multiparameter persistent homology landscapes identify immune cell spatial patterns in tumors' by Vipond et al.   
https://doi.org/10.1073/pnas.2102166118

Download the original dataset from https://github.com/MultiparameterTDAHistology/SpatialPatterningOfImmuneCells

Alternatively, precomputed ECCs and ECPs can be found as pickle files at  
https://drive.google.com/drive/folders/1RvGSG0TffSxbojCzuUWXz6jQeTn3RpOB?usp=sharing

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import glob

from tqdm.notebook import tqdm

In [ ]:
EPSILON = 0.2
KNN = 10

POINTS_SCALE = 600
CODENSITY_SCALE = 360

## Read data

In [ ]:
!unzip 1.5mmRegions.zip -d .

In [ ]:
data_15 = {}

for folder in ['CD8', 'CD68', 'FoxP3']:
    for filename in glob.glob('1.5mmRegions/{}/'.format(folder)+'*.csv'):
        data_15[filename.split('/')[-1]] = pd.read_csv(filename).values

# 1 dim VR ECC

In [ ]:
!git clone --recursive https://github.com/dgurnari/pyEulerCurves.git
!pip install ./pyEulerCurves -q

In [ ]:
from pyEulerCurves import ECC_from_pointcloud

In [ ]:
trans = ECC_from_pointcloud(epsilon=EPSILON, # max filtration
                            workers=1    # number of CPU cores
                           )


In [ ]:
for LABEL in tqdm(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'N', 'O']):

    ECC_global = {}

    for key in tqdm([k for k in data_15.keys() if 'T_{}'.format(LABEL) in k]):

        ecc = trans.fit_transform(data_15[key] / 600)
        ECC_global[key] = {'ecc': ecc}

    with open('pkls/ECC_global_{}_{}.pkl'.format(LABEL, EPSILON), 'wb') as f:
        pickle.dump(ECC_global, f)

# 1 dim alpha ECC

In [ ]:
!pip install gudhi -q

In [ ]:
from scipy.spatial import distance_matrix

def kNN_filter(points, kNN):
    D = distance_matrix(points, points)
    sortedD = np.sort(D)
    codensity = np.sum(sortedD[:, :kNN+1], axis=1)
    #filtered_points = np.hstack((points, np.expand_dims(codensity, axis=1)))
    #return filtered_points
    return codensity

In [ ]:
import gudhi as gd

def euler_characteristic_list_from_all(local_contributions):

    euler_characteristic = []
    old_f, current_characteristic = local_contributions[0]

    for filtration, contribution in local_contributions[1:]:
        if filtration > old_f:
            euler_characteristic.append([old_f, current_characteristic])
            old_f = filtration

        current_characteristic += contribution

    # add last contribution
    if len(local_contributions) > 1:
        euler_characteristic.append([filtration, current_characteristic])
        
    if len(local_contributions) == 1:
        euler_characteristic.append(local_contributions[0])

    return euler_characteristic



def compute_ECC_ECP_alpha(point_cloud, vertex_filtrations, dbg=False):
    alpha_complex = gd.AlphaComplex(points=point_cloud)
    simplex_tree = alpha_complex.create_simplex_tree()
    
    ecp = {}
    ecc = {}
    num_simplices = 0
    
    for s, f1 in simplex_tree.get_filtration():
        f2 = max([vertex_filtrations[i] for i in s])
        dim = len(s) - 1

        ecp[(f1, f2)] = ecp.get((f1, f2), 0) + (-1)**dim
        ecc[f1] = ecc.get(f1, 0) + (-1)**dim

        num_simplices += 1
        
        if dbg: 
            print(s, f1, f2)
        
    return (sorted(list(ecc.items()), key = lambda x: x[0]),
            sorted(list(ecp.items()), key = lambda x: x[0]), num_simplices)

In [ ]:
for LABEL in tqdm(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'N', 'O']):

    ECC_ECP_alpha = {}

    for key in tqdm([k for k in data_15.keys() if 'T_{}'.format(LABEL) in k]):

        ecc_contr, ecp, num_simplices = compute_ECC_ECP_alpha(data_15[key] / 600, 
                                                        kNN_filter(data_15[key], KNN) / 360,
                                                        dbg=False
                                                        )
        ECC_ECP_alpha[key] = {'ecc': euler_characteristic_list_from_all(ecc_contr),
                              'ecp': ecp,
                              'num_simplices': num_simplices}

    with open('pkls/alpha/ECC_ECP_alpha_{}.pkl'.format(LABEL), 'wb') as f:
        pickle.dump(ECC_ECP_alpha, f)

# 2 dim ECP

In [ ]:
import src.purepyECP as purepyECP
from src.bifiltration_utils import EC_at_bifiltration, plot_ECP, difference_ECP

In [ ]:
from scipy.spatial import distance_matrix

def kNN_filter(points, kNN):
    D = distance_matrix(points, points)
    sortedD = np.sort(D)
    codensity = np.sum(sortedD[:, :kNN+1], axis=1)
    #filtered_points = np.hstack((points, np.expand_dims(codensity, axis=1)))
    #return filtered_points
    return codensity

In [ ]:
len(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'N', 'O'])

13

In [ ]:
for LABEL in tqdm(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'N', 'O']):

    ECP_global = {}

    for key in tqdm([k for k in data_15.keys() if 'T_{}'.format(LABEL) in k]):

        contributions, num_simplices = purepyECP.compute_local_contributions(data_15[key] / 600, 
                                                                            EPSILON, 
                                                                            kNN_filter(data_15[key], KNN) / 360,
                                                                            dbg=False
                                                                            )
        ECP_global[key] = {'contributions': contributions,
                            'num_simplices': num_simplices}

    with open('pkls/ECP_global_{}_{}.pkl'.format(LABEL, EPSILON), 'wb') as f:
        pickle.dump(ECP_global, f)

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/225 [00:00<?, ?it/s]